In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 17.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=c8d07b069b80365b40fd95f876c5e957c4bbaa830299e3220d60feb20f383f69
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [2]:
!pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Importing Libaries

In [4]:
import pyspark
import findspark
findspark.init()

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import *

Creating Spark Session

In [5]:
spark = SparkSession.builder.master("local").appName("HeartDisease").getOrCreate()

In [6]:
spark

Loading Data into dataframe

In [7]:
data = spark.read.csv("/content/heart.csv",inferSchema=True, header=True)

In [8]:
data.show()

+---+------+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+
|Age|Gender|ChestPainType|RestingBP|Cholesterol|FastingBS|RestingECG|MaxHR|ExerciseAngina|Oldpeak|ST_Slope|HeartDisease|
+---+------+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+
| 40|     M|          ATA|      140|        289|        0|    Normal|  172|             N|    0.0|      Up|           0|
| 49|     F|          NAP|      160|        180|        0|    Normal|  156|             N|    1.0|    Flat|           1|
| 37|     M|          ATA|      130|        283|        0|        ST|   98|             N|    0.0|      Up|           0|
| 48|     F|          ASY|      138|        214|        0|    Normal|  108|             Y|    1.5|    Flat|           1|
| 54|     M|          NAP|      150|        195|        0|    Normal|  122|             N|    0.0|      Up|           0|
| 39|     M|          NAP|      

In [9]:
data.printSchema()

root
 |-- Age: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- ChestPainType: string (nullable = true)
 |-- RestingBP: integer (nullable = true)
 |-- Cholesterol: integer (nullable = true)
 |-- FastingBS: integer (nullable = true)
 |-- RestingECG: string (nullable = true)
 |-- MaxHR: integer (nullable = true)
 |-- ExerciseAngina: string (nullable = true)
 |-- Oldpeak: double (nullable = true)
 |-- ST_Slope: string (nullable = true)
 |-- HeartDisease: integer (nullable = true)



In [10]:
data = data.withColumn('Age', col('Age').astype('float'))
data = data.withColumn('RestingBP', col('RestingBP').astype('float'))
data = data.withColumn('Cholesterol', col('Cholesterol').astype('float'))
data = data.withColumn('FastingBS', col('FastingBS').astype('float'))
data = data.withColumn('MaxHR', col('MaxHR').astype('float'))
data = data.withColumn('HeartDisease', col('HeartDisease').astype('float'))

In [11]:
data.printSchema()

root
 |-- Age: float (nullable = true)
 |-- Gender: string (nullable = true)
 |-- ChestPainType: string (nullable = true)
 |-- RestingBP: float (nullable = true)
 |-- Cholesterol: float (nullable = true)
 |-- FastingBS: float (nullable = true)
 |-- RestingECG: string (nullable = true)
 |-- MaxHR: float (nullable = true)
 |-- ExerciseAngina: string (nullable = true)
 |-- Oldpeak: double (nullable = true)
 |-- ST_Slope: string (nullable = true)
 |-- HeartDisease: float (nullable = true)



In [12]:
data.show()

+----+------+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+
| Age|Gender|ChestPainType|RestingBP|Cholesterol|FastingBS|RestingECG|MaxHR|ExerciseAngina|Oldpeak|ST_Slope|HeartDisease|
+----+------+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+
|40.0|     M|          ATA|    140.0|      289.0|      0.0|    Normal|172.0|             N|    0.0|      Up|         0.0|
|49.0|     F|          NAP|    160.0|      180.0|      0.0|    Normal|156.0|             N|    1.0|    Flat|         1.0|
|37.0|     M|          ATA|    130.0|      283.0|      0.0|        ST| 98.0|             N|    0.0|      Up|         0.0|
|48.0|     F|          ASY|    138.0|      214.0|      0.0|    Normal|108.0|             Y|    1.5|    Flat|         1.0|
|54.0|     M|          NAP|    150.0|      195.0|      0.0|    Normal|122.0|             N|    0.0|      Up|         0.0|
|39.0|     M|          N

Checking Dataframe size

In [13]:
# Counting the number of rows in the DataFrame
num_rows = data.count()

# Counting the number of columns in the DataFrame
num_cols = len(data.columns)

print("Number of rows: ", num_rows)
print("Number of columns: ", num_cols)

Number of rows:  918
Number of columns:  12


Statistical Summary Of Dataframe

In [14]:
data.describe().show()

+-------+------------------+------+-------------+------------------+------------------+-------------------+----------+------------------+--------------+------------------+--------+-------------------+
|summary|               Age|Gender|ChestPainType|         RestingBP|       Cholesterol|          FastingBS|RestingECG|             MaxHR|ExerciseAngina|           Oldpeak|ST_Slope|       HeartDisease|
+-------+------------------+------+-------------+------------------+------------------+-------------------+----------+------------------+--------------+------------------+--------+-------------------+
|  count|               918|   918|          918|               918|               918|                918|       918|               918|           918|               918|     918|                918|
|   mean|53.510893246187365|  null|         null|132.39651416122004| 198.7995642701525|0.23311546840958605|      null|136.80936819172112|          null|0.8873638344226581|    null| 0.5533769063180

Checking the duplicate records in data

In [15]:
data.distinct().count()

918

Checking Null Values in dataframe

In [16]:
null_counts = data.agg(*[count(when(col(c).isNull(), c)).alias(c) for c in data.columns])

# Display the null counts
null_counts.show()

+---+------+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+
|Age|Gender|ChestPainType|RestingBP|Cholesterol|FastingBS|RestingECG|MaxHR|ExerciseAngina|Oldpeak|ST_Slope|HeartDisease|
+---+------+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+
|  0|     0|            0|        0|          0|        0|         0|    0|             0|      0|       0|           0|
+---+------+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+



In [17]:
data.groupBy("ChestPainType").count().show()

+-------------+-----+
|ChestPainType|count|
+-------------+-----+
|          NAP|  203|
|          ATA|  173|
|           TA|   46|
|          ASY|  496|
+-------------+-----+



In [18]:
data.groupBy("RestingECG").count().show()

+----------+-----+
|RestingECG|count|
+----------+-----+
|       LVH|  188|
|    Normal|  552|
|        ST|  178|
+----------+-----+



In [19]:
data.groupBy("ST_Slope").count().show()

+--------+-----+
|ST_Slope|count|
+--------+-----+
|    Flat|  460|
|      Up|  395|
|    Down|   63|
+--------+-----+



In [20]:
data.groupBy("HeartDisease").count().show()

+------------+-----+
|HeartDisease|count|
+------------+-----+
|         1.0|  508|
|         0.0|  410|
+------------+-----+



1 Means person has Heart Disease

0 Means Person doesn't have Heart Disease

Handling the Catogorical Features

In [21]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler

In [22]:
indexer = StringIndexer(inputCols=["Gender","ChestPainType","RestingECG","ExerciseAngina","ST_Slope"],
                        outputCols=["Gender_cat","ChestPainType_cat","RestingECG_cat","ExerciseAngina_cat","ST_Slope_cat"])
indexed = indexer.fit(data).transform(data)

In [23]:
indexed.show()

+----+------+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+----------+-----------------+--------------+------------------+------------+
| Age|Gender|ChestPainType|RestingBP|Cholesterol|FastingBS|RestingECG|MaxHR|ExerciseAngina|Oldpeak|ST_Slope|HeartDisease|Gender_cat|ChestPainType_cat|RestingECG_cat|ExerciseAngina_cat|ST_Slope_cat|
+----+------+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+----------+-----------------+--------------+------------------+------------+
|40.0|     M|          ATA|    140.0|      289.0|      0.0|    Normal|172.0|             N|    0.0|      Up|         0.0|       0.0|              2.0|           0.0|               0.0|         1.0|
|49.0|     F|          NAP|    160.0|      180.0|      0.0|    Normal|156.0|             N|    1.0|    Flat|         1.0|       1.0|              1.0|           0.0|               0.0|         0.0|
|37.0|    

Used VectorAssembler to combine all Independent variables or features

In [24]:
assembler = VectorAssembler(inputCols=['Age',
 'RestingBP',
 'Cholesterol',
 'FastingBS',
 'MaxHR',
 'Oldpeak',
 'Gender_cat', 'ChestPainType_cat', 'RestingECG_cat',
 'ExerciseAngina_cat',
 'ST_Slope_cat'],outputCol='features')

In [25]:
assembler

VectorAssembler_99b00e64bc05

In [26]:
output = assembler.transform(indexed)

output.show(truncate=False)

+----+------+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+----------+-----------------+--------------+------------------+------------+----------------------------------------------------+
|Age |Gender|ChestPainType|RestingBP|Cholesterol|FastingBS|RestingECG|MaxHR|ExerciseAngina|Oldpeak|ST_Slope|HeartDisease|Gender_cat|ChestPainType_cat|RestingECG_cat|ExerciseAngina_cat|ST_Slope_cat|features                                            |
+----+------+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+----------+-----------------+--------------+------------------+------------+----------------------------------------------------+
|40.0|M     |ATA          |140.0    |289.0      |0.0      |Normal    |172.0|N             |0.0    |Up      |0.0         |0.0       |2.0              |0.0           |0.0               |1.0         |(11,[0,1,2,4,7,10],[40.0,140.0,289.0,172.0,2.0,1.0

Creating New DataFrame which contains only Independent and Dependent features

In [27]:
model = output.select('features','HeartDisease')

In [28]:
model.show()

+--------------------+------------+
|            features|HeartDisease|
+--------------------+------------+
|(11,[0,1,2,4,7,10...|         0.0|
|[49.0,160.0,180.0...|         1.0|
|[37.0,130.0,283.0...|         0.0|
|[48.0,138.0,214.0...|         1.0|
|(11,[0,1,2,4,7,10...|         0.0|
|(11,[0,1,2,4,7,10...|         0.0|
|[45.0,130.0,237.0...|         0.0|
|(11,[0,1,2,4,7,10...|         0.0|
|(11,[0,1,2,4,5,9]...|         1.0|
|[48.0,120.0,284.0...|         0.0|
|[37.0,130.0,211.0...|         0.0|
|[58.0,136.0,164.0...|         1.0|
|(11,[0,1,2,4,7,10...|         0.0|
|(11,[0,1,2,4,5,9]...|         1.0|
|[42.0,115.0,211.0...|         0.0|
|[54.0,120.0,273.0...|         0.0|
|(11,[0,1,2,4],[38...|         1.0|
|[43.0,120.0,201.0...|         0.0|
|(11,[0,1,2,4,5],[...|         1.0|
|(11,[0,1,2,4,5,7]...|         1.0|
+--------------------+------------+
only showing top 20 rows



Split Train and Test Data

In [29]:
train_data, test_data = model.randomSplit([0.7,0.3])

In [30]:
print(train_data.count(),test_data.count())

642 276


Logistic Regression

In [31]:
from pyspark.ml.classification import LogisticRegression

In [32]:
#lr = LogisticRegression(labelCol='HeartDisease').fit(train_data)

lr = LogisticRegression(featuresCol='features',labelCol='HeartDisease').fit(train_data)

In [33]:
lr

LogisticRegressionModel: uid=LogisticRegression_4c6862ee92b1, numClasses=2, numFeatures=11

In [34]:
train_result = lr.evaluate(train_data).predictions

In [35]:
train_result.show()

+--------------------+------------+--------------------+--------------------+----------+
|            features|HeartDisease|       rawPrediction|         probability|prediction|
+--------------------+------------+--------------------+--------------------+----------+
|(11,[0,1,2,3,4],[...|         1.0|[-1.6094177306173...|[0.16666946971562...|       1.0|
|(11,[0,1,2,3,4,5]...|         1.0|[-3.3107157463934...|[0.03520540008898...|       1.0|
|(11,[0,1,2,3,4,5]...|         1.0|[-4.6816403464470...|[0.00917877517788...|       1.0|
|(11,[0,1,2,3,4,7]...|         1.0|[-1.6619404411531...|[0.15950168774851...|       1.0|
|(11,[0,1,2,3,4,7]...|         1.0|[-1.9482861774663...|[0.12474035336261...|       1.0|
|(11,[0,1,2,3,4,8]...|         1.0|[-2.1315836578703...|[0.10606474343034...|       1.0|
|(11,[0,1,2,3,4,8]...|         1.0|[-1.9919462429616...|[0.12005111153245...|       1.0|
|(11,[0,1,2,3,4,8]...|         1.0|[-2.8779031755184...|[0.05325676030919...|       1.0|
|(11,[0,1,2,4],[32...

In [36]:
lrc = lr.evaluate(test_data).predictions

In [37]:
lrc.show()

+--------------------+------------+--------------------+--------------------+----------+
|            features|HeartDisease|       rawPrediction|         probability|prediction|
+--------------------+------------+--------------------+--------------------+----------+
|(11,[0,1,2,3,4,5]...|         1.0|[-1.8890460138772...|[0.13135328059459...|       1.0|
|(11,[0,1,2,3,4,8]...|         1.0|[-2.2150757424366...|[0.09840482777418...|       1.0|
|(11,[0,1,2,4],[39...|         1.0|[0.39979604154952...|[0.59863865591684...|       0.0|
|(11,[0,1,2,4],[46...|         1.0|[-0.0028049954364...|[0.49929875160066...|       1.0|
|(11,[0,1,2,4],[50...|         1.0|[-0.1107709031819...|[0.47233555576049...|       1.0|
|(11,[0,1,2,4],[52...|         1.0|[-0.3260225673161...|[0.41920870733709...|       1.0|
|(11,[0,1,2,4],[59...|         1.0|[-0.9722511110161...|[0.27443203549314...|       1.0|
|(11,[0,1,2,4,5],[...|         1.0|[-1.8132879750067...|[0.14024121355857...|       1.0|
|(11,[0,1,2,4,5],[...

In [38]:
lrc.select(["HeartDisease", "prediction"]).show(10)

+------------+----------+
|HeartDisease|prediction|
+------------+----------+
|         1.0|       1.0|
|         1.0|       1.0|
|         1.0|       0.0|
|         1.0|       1.0|
|         1.0|       1.0|
|         1.0|       1.0|
|         1.0|       1.0|
|         1.0|       1.0|
|         1.0|       1.0|
|         0.0|       0.0|
+------------+----------+
only showing top 10 rows



Calculate confusion matrix

In [40]:
tp = lrc[(lrc.HeartDisease==1) & (lrc.prediction==1)].count()
tn = lrc[(lrc.HeartDisease==0) & (lrc.prediction==0)].count()
fp = lrc[(lrc.HeartDisease==0) & (lrc.prediction==1)].count()
fn = lrc[(lrc.HeartDisease==1) & (lrc.prediction==0)].count()

confusion_matrix = [[tn, fp], [fn, tp]]

In [41]:
accuracy = float((tp+tn) / (lrc.count()))
recal = float(tn)/(tp+tn) 
precision = float(tp)/(tp+fp) 
f1_score = float(2*(precision*recal) / (precision+recal)) 

In [42]:
print("Confusion Matrix: ")
print(confusion_matrix)
print("")
print("----Classification Report Of Logistic Regression----")
print("-----------------------------------------------------")
print("Accuracy Of Logistic Regression Is:",accuracy)

print("Recall of Logistic Regression Is:",recal)

print("Precision of Logistic Regression Is:",precision)

print("F1_Score of Logistic Regression Is:",precision)
print("-----------------------------------------------------")

Confusion Matrix: 
[[108, 15], [17, 136]]

----Classification Report Of Logistic Regression----
-----------------------------------------------------
Accuracy Of Logistic Regression Is: 0.8840579710144928
Recall of Logistic Regression Is: 0.4426229508196721
Precision of Logistic Regression Is: 0.9006622516556292
F1_Score of Logistic Regression Is: 0.9006622516556292
-----------------------------------------------------


Pickel The Model

In [45]:
pic_model = lrc.toPandas()

pic_model.head(10)

,features,HeartDisease,rawPrediction,probability,prediction
0,"(67.0, 125.0, 254.0, 1.0, 163.0, 0.2, 0.0, 0.0...",1.0,"[-1.8890460138772787, 1.8890460138772787]","[0.13135328059459547, 0.8686467194054045]",1.0
1,"(55.0, 116.0, 186.0, 1.0, 102.0, 0.0, 0.0, 0.0...",1.0,"[-2.2150757424366456, 2.2150757424366456]","[0.09840482777418617, 0.9015951722258139]",1.0
2,"(39.0, 110.0, 280.0, 0.0, 150.0, 0.0, 0.0, 0.0...",1.0,"[0.39979604154952797, -0.39979604154952797]","[0.5986386559168457, 0.40136134408315427]",0.0
3,"(46.0, 134.0, 310.0, 0.0, 126.0, 0.0, 0.0, 0.0...",1.0,"[-0.0028049954364863794, 0.0028049954364863794]","[0.49929875160066356, 0.5007012483993365]",1.0
4,"(50.0, 145.0, 264.0, 0.0, 150.0, 0.0, 0.0, 0.0...",1.0,"[-0.11077090318199989, 0.11077090318199989]","[0.4723355557604901, 0.52766444423951]",1.0
5,"(52.0, 120.0, 182.0, 0.0, 150.0, 0.0, 0.0, 0.0...",1.0,"[-0.32602256731611456, 0.32602256731611456]","[0.41920870733709287, 0.5807912926629071]",1.0
6,"(59.0, 130.0, 126.0, 0.0, 125.0, 0.0, 0.0, 0.0...",1.0,"[-0.9722511110161218, 0.9722511110161218]","[0.274432035493144, 0.725567964506856]",1.0
7,"(54.0, 140.0, 216.0, 0.0, 105.0, 1.5, 0.0, 0.0...",1.0,"[-1.8132879750067972, 1.8132879750067972]","[0.14024121355857133, 0.8597587864414287]",1.0
8,"(67.0, 120.0, 237.0, 0.0, 71.0, 1.0, 0.0, 0.0,...",1.0,"[-1.911252617591412, 1.911252617591412]","[0.12884019247285325, 0.8711598075271467]",1.0
9,"(58.0, 130.0, 197.0, 0.0, 131.0, 0.6, 1.0, 0.0...",0.0,"[0.13848512881844244, -0.13848512881844244]","[0.5345660571566646, 0.4654339428433354]",0.0


In [46]:
import pickle

In [47]:
with open('lr.pkl', 'wb') as f:
    pickle.dump(pic_model, f)